In [1]:
# Phong_Thesis

# <center>Hệ thống tự động nhận diện trạng thái bãi đỗ xe</center>

## Lý thuyết
Hệ thống được hình thành để phân loại các lot đậu xe thành 2 loại là trống hoặc đã có xe đỗ vào. Hệ thống này đặc biệt tối ưu với các bãi đỗ xe có camera ở các cột đèn hay có tầm nhìn thoáng và rộng. Hệ thống sử dụng sự kết hợp của toán tử Laplace để xác định viền ảnh, Haar classifier để nhận dạng đối tượng và theo dõi chuyển động để phân biệt giữa chỗ chưa đã có và chưa có xe đậu. Theo dõi chuyển động được thiết kế bằng việc sử dụng kĩ thuật background substraction (tách cảnh nền), contours (được hiểu đơn giản là một đường cong liên kết toàn bộ các điểm liên tục (dọc theo đường biên) mà có cùng màu sắc hoặc giá trị cường độ.) và Morphological operations cũng được sử dụng. 


## Hướng tiếp cận
Haar classifier đã được cân nhắc so sánh với SVM và HOG. Mặc dù HOG kết hợp với SVM sẽ mang lại độ chính xác cao hơn nếu so sánh với Haar nhưng việc các yêu cầu về quá trình xử lý của Haar là ít hơn rất nhiều so với HOG. Cuối cùng, Haar được lựa chọn để áp dụng vào hệ thống lần này.Chức năng phát hiện người qua đường của HOG được cài đặt sẵn trong opencv2 cũng được sử dụng nhưng nó được tắt đi vì nó sẽ làm chậm tốc độ xử lý của hệ thống. Để tăng độ chính xác trong việc phân loại các lot đỗ xe, toán tử Laplace được sử dụng. Threshold sẽ được thiết lập nơi khả năng hiện diện của xe là tối đa. Về cơ bản, các khu vực đậu xe xác định bằng tay và threshold được xác định bằng cách tính toán độ lớn của các cạnh bên trong chúng. Với một chiếc xe thì nó sẽ có một số cạnh đáng kể, trong khi chỗ đậu xe trống gần như là mịn màng. Đây là thực tế đằng sau việc sử dụng toán tử Laplace. Nếu khả năng hiện diện của một chiếc xe vượt quá threshold thì thuật toán phân loại ngay lập tức được áp dụng để phát hiện liệu nó có phải là một chiếc xe hay không. Bất cứ khi nào một sự thay đổi threshold được tìm thấy, thuật toán phân loại sẽ được gọi là để kiểm tra liệu đó là một chiếc xe hay không. Nếu được xác định là một chiếc xe thì nó sẽ được đánh dấu và quan sát chuyển động. Bất cứ khi nào có một chuyển động tại một điểm, thuật toán phân loại sẽ lại được gọi là để xác định sự hiện diện của xe. Vì vậy hệ thống đề xuất một sự kết hợp của phát hiện, theo dõi và phân loại để đạt được bãi đậu xe hiệu quả.     

## Kết quả
Một chương trình đã được em sử dụng để tính toán độ chính xác của thuật toán phân loại trong việc phát hiện xe. Chương trình áp dụng thuật toán phân loại vào một số khung hình được em cắt ra từ video và từ đó tính toán được tỉ lệ phát hiện chính xác số xe trong từng khung hình. Tiếp đó, nó sẽ hiển thị giá trị trung bình tỉ lệ của tất cả các khung hình. Kết quả thu được như sau:

* Classifier được train với: **1562 Positives and 1772 Negatives** (có thể kiểm tra bằng việc click vào file 'haarTraining.bat'. Có thể kiểm tra ở mục screenshots để xem quá trình training)
* Số khung hình được sử dụng: **5**
* Trung bình tỉ lệ chính xác:  **62.25%** (Xem phần tính toán bằng đoạn code được viết ở cuối cùng)


Threshold được thay đổi và em nhận thấy rằng nó sẽ cho ra tỉ lệ chính xác nhật với giá trị là 2.7 (Vui lòng kiểm tra chương trình tính toán giá trị Threshold)


Kết quả của thí nghiệm kiểm tra Threshold với tổng số xe là 120

* Threshold = 1.5  Số xe không được phân loại = 54 

* Threshold = 2.0  Số xe không được phân loại = 39 

* Threshold = 2.8  Số xe không được phân loại = 19

* Threshold = 3.0  Số xe không được phân loại = 24

* Threshold = 3.5  Số xe không được phân loại = 38

## Code
1. Chương trình chính của hệ thống.
2. Chương trình cho phép ta thấy độ nhạy của thuật toán áp dụng đối với đường cao tốc M6, một trong những đường cao tốc có lưu lượng xe lớn nhất thế giới.
3. Chương trình cho phép ta xác định độ chính xác của thuật toán phân loại
4. Chương trình cho phép ta xác định các lot đỗ xe trên video bằng tay và lưu chúng vào yaml file để phục vụ cho hệ thống.
5. Chương trình giúp tính toán giá trị ước lượng của Threshold cho một video xác định.

In [8]:
"""
Hướng dẫn: Escape key Nhấn Esc để thoát chương trình, lưu ý có thể ấn nhiều lần nếu lệnh không được thực hiện.
Nhấn U để nhảy 500 frames và nhận J để nhảy 1000 frames.
Các giá trị có thể chỉnh sửa được trong dictionary:
1. show_ids: Bật hoặc tắt id cuẩ từng lot
2. save_video: Lưu video được tạo ra từ chương trình
3. text_overlay: Hiển thị đếm frame ở góc trên màn hình.
4. motion_detection: Bật hoặc tắt motion detection
5. pedestrian detection: Khiến cho chương trình chậm vì function HOG trong opencv
6. min_area_motion_contour: Area cần nhỏ nhất cho motion
7. start_frame: Bắt đầu từ frame nào
8. park_laplacian_th: Thiết lập giá trị Threshold
"""

import yaml
import numpy as np
import cv2


fn = "myCam.avi" #3
fn_yaml = "phong_yml_01.yml"
fn_out =  "phong_outputvideo_02.avi"
cascade_src = 'cars.xml'
car_cascade = cv2.CascadeClassifier(cascade_src)
global_str = "Last change at: "
change_pos = 0.00
dict =  {
        'print_out':True,
        'text_overlay': True,
        'parking_overlay': True,
        'parking_id_overlay': True,
        'parking_detection': True,
        'motion_detection': False,
        'pedestrian_detection': False, # mất nhiều power
        'min_area_motion_contour': 500, 
        'park_laplacian_th': 1.9, 
        'park_sec_to_wait': 1, #thời gian đợi để thay đổi trạng thái của region 
        'start_frame': 0, # Bắt đầu từ frame nào
        'show_ids': True, # Hiển thị id cho từng lot
        'classifier_used': True,
        'save_video': False
        }

# Set từ video
cap = cv2.VideoCapture(fn)
video_info = {  'fps':    cap.get(cv2.CAP_PROP_FPS),
                'width':  int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)*0.6),
                'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)*0.6),
                'fourcc': cap.get(cv2.CAP_PROP_FOURCC),
                'num_of_frames': int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}

cap.set(cv2.CAP_PROP_POS_FRAMES, dict['start_frame']) # Nhảy đến frame được xác định trước

def run_classifier(img, id):
    cars = car_cascade.detectMultiScale(img, 1.1, 1)
    if cars == ():
        return False
    else:
        return True

# Định nghĩa codec và tạo VideoWriter object
if dict['save_video']:
    fourcc = cv2.VideoWriter_fourcc('X','V','I','D') # các lựa chọn: ('P','I','M','1'), ('D','I','V','X'), ('M','J','P','G'), ('X','V','I','D')
    out = cv2.VideoWriter(fn_out, -1, 25.0,(video_info['width'], video_info['height']))

# Khởi tạo HOG descriptor/person detector. Mất rất nhiều power cho quá trình này.
if dict['pedestrian_detection']:
    hog = cv2.HOGDescriptor()
    hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

    # Sử dụng Background subtraction tách cảnh nền.
if dict['motion_detection']:
    fgbg = cv2.createBackgroundSubtractorMOG2(history=300, varThreshold=16, detectShadows=True)

# Đọc file yaml (parking space polygons)
with open(fn_yaml, 'r') as stream:
    parking_data = yaml.load(stream)
parking_contours = []
parking_bounding_rects = []
parking_mask = []
parking_data_motion = []
if parking_data != None:
    for park in parking_data:
        points = np.array(park['points'])
        rect = cv2.boundingRect(points)
        points_shifted = points.copy()
        points_shifted[:,0] = points[:,0] - rect[0] # shift contour to region of interest
        points_shifted[:,1] = points[:,1] - rect[1]
        parking_contours.append(points)
        parking_bounding_rects.append(rect)
        mask = cv2.drawContours(np.zeros((rect[3], rect[2]), dtype=np.uint8), [points_shifted], contourIdx=-1,
                                    color=255, thickness=-1, lineType=cv2.LINE_8)
        mask = mask==255
        parking_mask.append(mask)

kernel_erode = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)) # morphological kernel
kernel_dilate = cv2.getStructuringElement(cv2.MORPH_RECT,(5,19))
if parking_data != None:
    parking_status = [False]*len(parking_data)
    parking_buffer = [None]*len(parking_data)

def print_parkIDs(park, coor_points, frame_rev):
    moments = cv2.moments(coor_points)
    centroid = (int(moments['m10']/moments['m00'])-3, int(moments['m01']/moments['m00'])+3)
    # Gắn số vào các region được marked bằng tay
    cv2.putText(frame_rev, str(park['id']), (centroid[0]+1, centroid[1]+1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    cv2.putText(frame_rev, str(park['id']), (centroid[0]-1, centroid[1]-1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    cv2.putText(frame_rev, str(park['id']), (centroid[0]+1, centroid[1]-1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    cv2.putText(frame_rev, str(park['id']), (centroid[0]-1, centroid[1]+1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1, cv2.LINE_AA)
    cv2.putText(frame_rev, str(park['id']), centroid, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)

while(cap.isOpened()):
    current_count = 0
    video_cur_pos = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0 # Vị trí hiện tại của video file tính theo giây
    video_cur_frame = cap.get(cv2.CAP_PROP_POS_FRAMES) # Vị trí tính theo frame
    ret, frame_initial = cap.read()
    if ret == True:
        frame = cv2.resize(frame_initial, None, fx=0.6, fy=0.6)
    if ret == False:
        print("Video ended")
        break

    # Background Subtraction
    frame_blur = cv2.GaussianBlur(frame.copy(), (5,5), 3)
    frame_gray = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY)
    frame_out = frame.copy()

    # Hiển thị số frame trên góc trái video
    if dict['text_overlay']:
        str_on_frame = "%d/%d" % (video_cur_frame, video_info['num_of_frames'])
        cv2.putText(frame_out, str_on_frame, (5,30), cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (0,255,255), 2, cv2.LINE_AA)
        cv2.putText(frame_out,global_str + str(round(change_pos,2)) + 'sec', (5, 60), cv2.FONT_HERSHEY_SIMPLEX,
                        0.8, (255, 0, 0), 2, cv2.LINE_AA)

    # motion detection cho mọi objects
    if dict['motion_detection']:
        fgmask = fgbg.apply(frame_blur)
        bw = np.uint8(fgmask==255)*255
        bw = cv2.erode(bw, kernel_erode, iterations=1)
        bw = cv2.dilate(bw, kernel_dilate, iterations=1)
        (_, cnts, _) = cv2.findContours(bw.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # Áp loop cho contours
        for c in cnts:
            # Nếu contours quá nhỏ thì bỏ qua
            if cv2.contourArea(c) < dict['min_area_motion_contour']:
                continue
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(frame_out, (x, y), (x + w, y + h), (255, 0, 0), 1)

    # Detect xe và chỗ trống
    if dict['parking_detection']:
        for ind, park in enumerate(parking_data):
            points = np.array(park['points'])
            rect = parking_bounding_rects[ind]
            roi_gray = frame_gray[rect[1]:(rect[1]+rect[3]), rect[0]:(rect[0]+rect[2])] # crop ROI để tính toán nhanh hơn

            laplacian = cv2.Laplacian(roi_gray, cv2.CV_64F)
            points[:,0] = points[:,0] - rect[0] # Chuyển contour sang ROI
            points[:,1] = points[:,1] - rect[1]
            delta = np.mean(np.abs(laplacian * parking_mask[ind]))
            status = delta < dict['park_laplacian_th']
            # Nếu phát hiện có sự thay đổi thì lưu thời gian lại
            if status != parking_status[ind] and parking_buffer[ind]==None:
                parking_buffer[ind] = video_cur_pos
                change_pos = video_cur_pos
    
            # Nếu trạng thái vẫn khác với cái đã được lưu và counter đang open
            elif status != parking_status[ind] and parking_buffer[ind]!=None:
                if video_cur_pos - parking_buffer[ind] > dict['park_sec_to_wait']:
                    parking_status[ind] = status
                    parking_buffer[ind] = None
            # Nếu trạng thái vẫn như vậy và counter đang open
            elif status == parking_status[ind] and parking_buffer[ind]!=None:
                parking_buffer[ind] = None

    # Thay đổi màu và trạng thái hiển thị trên section phía trên 
    if dict['parking_overlay']:
        for ind, park in enumerate(parking_data):
            points = np.array(park['points'])
            if parking_status[ind]:
                color = (0,255,0)
                rect = parking_bounding_rects[ind]
                roi_gray_ov = frame_gray[rect[1]:(rect[1] + rect[3]),
                               rect[0]:(rect[0] + rect[2])]  # crop ROI để tính toán nhanh hơn
                res = run_classifier(roi_gray_ov, ind)
                current_count += 1
                if res:
                    parking_data_motion.append(parking_data[ind])
                    color = (0,0,255)
            else:
                color = (0,0,255)
                
            
            cv2.drawContours(frame_out, [points], contourIdx=-1,
                                 color=color, thickness=2, lineType=cv2.LINE_8)
            
            if dict['show_ids']:
                    print_parkIDs(park, points, frame_out)
        #Hiển thị số lot trống trong frame
        cv2.putText(frame_out,'Total vacant spots in frame: ' + str(current_count),(7,85),cv2.FONT_HERSHEY_SIMPLEX,0.728,(98,189,184),2,cv2.LINE_AA)
    
    if dict['print_out']:
        count=current_count
        file = open("test.txt", "w") 
        file.write(str(count)) 
        file.close() 
        
    if parking_data_motion != []:
        for index, park_coord in enumerate(parking_data_motion):
            points = np.array(park_coord['points'])
            color = (0, 0, 255)
            recta = parking_bounding_rects[ind]
            roi_gray1 = frame_gray[recta[1]:(recta[1] + recta[3]),
                            recta[0]:(recta[0] + recta[2])]  # crop ROI để tính toán nhanh hơn
            fgbg1 = cv2.createBackgroundSubtractorMOG2(history=300, varThreshold=16, detectShadows=True)
            roi_gray1_blur = cv2.GaussianBlur(roi_gray1.copy(), (5, 5), 3)
            fgmask1 = fgbg1.apply(roi_gray1_blur)
            bw1 = np.uint8(fgmask1 == 255) * 255
            bw1 = cv2.erode(bw1, kernel_erode, iterations=1)
            bw1 = cv2.dilate(bw1, kernel_dilate, iterations=1)
            (_, cnts1, _) = cv2.findContours(bw1.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # Áp loop cho contours
            for c in cnts1:
                print(cv2.contourArea(c))
                # Nếu contours quá nhỏ thì bỏ qua
                if cv2.contourArea(c) < 4:
                    continue
                (x, y, w, h) = cv2.boundingRect(c)
                classifier_result1 = run_classifier(roi_gray1, index)
                if classifier_result1:
                    color = (0, 0, 255)  # Đỏ nếu có xe
                else:
                    color = (0,255, 0)
            classifier_result1 = run_classifier(roi_gray1, index)
            if classifier_result1:
                color = (0, 0, 255)  # Đỏ nếu có xe
            else:
                color = (0, 255, 0)
            #cv2.drawContours(frame_out, [points], contourIdx=-1, color=color, thickness=2, lineType=cv2.LINE_8)
           
    if dict['pedestrian_detection']:
        # Detecr người trong video, sẽ làm giản tốc độ chương trình vì cần một GPU có tốc độ xử lý cao
        (rects, weights) = hog.detectMultiScale(frame, winStride=(4, 4), padding=(8, 8), scale=1.05)
        # Vẽ bounding box
        for (x, y, w, h) in rects:
            cv2.rectangle(frame_out, (x, y), (x + w, y + h), (255, 0, 0), 2)

    # write output frames
    if dict['save_video']:
            out.write(frame_out)

    # Hiển thị video
    cv2.imshow('frame', frame_out)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
    elif k == ord('c'):
        cv2.imwrite('frame%d.jpg' % video_cur_frame, frame_out)
    elif k == ord('j'):
        cap.set(cv2.CAP_PROP_POS_FRAMES, video_cur_frame+1000) # Nhảy 1000 frames
    elif k == ord('u'):
        cap.set(cv2.CAP_PROP_POS_FRAMES, video_cur_frame + 50)  # Nhảy 500 frames
    if cv2.waitKey(33) == 27:
        break

cv2.waitKey(0)
cap.release()
if dict['save_video']: out.release()
cv2.destroyAllWindows()   

<ipython-input-8-c5dd0a02c8be>:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parking_data = yaml.load(stream)
<ipython-input-8-c5dd0a02c8be>:56: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if cars == ():


In [7]:
# Classifier demonstration on M6 highway Britain:
import cv2
from imutils.object_detection import non_max_suppression

def perform_classification(video_src, cascade_src):
    cap = cv2.VideoCapture(video_src)
    car_cascade = cv2.CascadeClassifier(cascade_src)
    while True:
        ret, img = cap.read()
        if (type(img) == type(None)):
            print('Video not found')
            break
        image_scaled = cv2.resize(img, None, fx=0.6, fy=0.6)
        gray = cv2.cvtColor(image_scaled, cv2.COLOR_BGR2GRAY)
        cars = car_cascade.detectMultiScale(gray, 1.1, 1) #1.1, 1
        cars = np.array([[x, y, x + w, y + h] for (x, y, w, h) in cars])
        pick = non_max_suppression(cars, probs=None, overlapThresh=0.65)
        for (x, y, w, h) in pick:
            cv2.rectangle(image_scaled, (x, y), (w,  h), (0, 255, 255), 2)
        cv2.imshow('Press ESC key to finish', image_scaled)

            # Nhấn Esc để thoát
        if cv2.waitKey(33) == 27:
            break
    print('Execution finished')
    cv2.destroyAllWindows()
perform_classification('myCam.avi', 'cars.xml')    # M6 highway Britain



Execution finished


In [8]:
# Tính toán độ chính xác của classifier. Frames được lấy từ các buổi trong ngày với điều kiện ánh sáng khác sau (sáng chiều tối)
import phong_utility_01 as util
util.get_acc('phong_classifier.xml')

86 vehicles found
95 vehicles found
111 vehicles found
89 vehicles found
104 vehicles found
Accuracy after evaluating 5 frames and assuming correct identification:  62.25744531477015


62.25744531477015

In [2]:
# Nhấn Esc để hoàn thành việc xác định real time boxing
# Chương trình sẽ vẽ những hình tứ giác khi có đủ 4 lần nháy đúp chuột trái
import cv2
import yaml
import numpy as np

refPt = []
cropping = False
data = []
file_path = 'phong_yml_02.yml'
img = cv2.imread('phong_frame_02.png')

def yaml_loader(file_path):
    with open(file_path, "r") as file_descr:
        data = yaml.load(file_descr)
        return data


def yaml_dump(file_path, data):
    with open(file_path, "a") as file_descr:
        yaml.dump(data, file_descr)


def yaml_dump_write(file_path, data):
    with open(file_path, "w") as file_descr:
        yaml.dump(data, file_descr)


def click_and_crop(event, x, y, flags, param):
    current_pt = {'id': 0, 'points': []}
    # grab references cho biến global
    global refPt, cropping
    if event == cv2.EVENT_LBUTTONDBLCLK:
        refPt.append((x, y))
        cropping = False
    if len(refPt) == 4:
        if data == []:
            if yaml_loader(file_path) != None:
                data_already = len(yaml_loader(file_path))
            else:
                data_already = 0
        else:
            if yaml_loader(file_path) != None:
                data_already = len(data) + len(yaml_loader(file_path))
            else:
                data_already = len(data) 
        
        cv2.line(image, refPt[0], refPt[1], (0, 255, 0), 1)
        cv2.line(image, refPt[1], refPt[2], (0, 255, 0), 1)
        cv2.line(image, refPt[2], refPt[3], (0, 255, 0), 1)
        cv2.line(image, refPt[3], refPt[0], (0, 255, 0), 1)

        temp_lst1 = list(refPt[2])
        temp_lst2 = list(refPt[3])
        temp_lst3 = list(refPt[0])
        temp_lst4 = list(refPt[1])

        current_pt['points'] = [temp_lst1, temp_lst2, temp_lst3, temp_lst4]
        current_pt['id'] = data_already
        data.append(current_pt)
        refPt = []
image = cv2.resize(img, None, fx=0.6, fy=0.6)
clone = image.copy()
cv2.namedWindow("Double click to mark points")
cv2.imshow("Double click to mark points", image)
cv2.setMouseCallback("Double click to mark points", click_and_crop)

# Tiếp tục looping cho đến khi 'q' được nhấn
while True:
    # Hiển thị hình ảnh cho đến khi được được nhấn
    cv2.imshow("Double click to mark points", image)
    key = cv2.waitKey(1) & 0xFF
    if cv2.waitKey(33) == 27:
        break
       
# data list vào yaml file
if data != []:
    yaml_dump(file_path, data)
cv2.destroyAllWindows() 

In [16]:
# Chương trình tính toán giá trị Threshold cho một parking area có sẵn
import statistics
import cv2
import yaml
import numpy as np
        
sum_up = 0.0
delta_list = []
frame = cv2.imread('123.png') #
parking_bounding_rects = []
parking_mask = []
frame_blur = cv2.GaussianBlur(frame.copy(), (5,5), 3)
frame_gray = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2GRAY)
with open('phong_yml_01.yml', 'r') as stream:
    parking_data = yaml.load(stream)
    
if parking_data != None:
    for park in parking_data:
        points = np.array(park['points'])
        rect = cv2.boundingRect(points)
        points_shifted = points.copy()
        points_shifted[:,0] = points[:,0] - rect[0] # shift contour to region of interest
        points_shifted[:,1] = points[:,1] - rect[1]
        
        parking_bounding_rects.append(rect)
        mask = cv2.drawContours(np.zeros((rect[3], rect[2]), dtype=np.uint8), [points_shifted], contourIdx=-1,
                                    color=255, thickness=-1, lineType=cv2.LINE_8)
        mask = mask==255
        parking_mask.append(mask)

for ind, park in enumerate(parking_data):
        points = np.array(park['points'])
        rect = parking_bounding_rects[ind]
        roi_gray = frame_gray[rect[1]:(rect[1]+rect[3]), rect[0]:(rect[0]+rect[2])] # crop ROI để tính toán nhanh hơn

        laplacian = cv2.Laplacian(roi_gray, cv2.CV_64F)
        points[:,0] = points[:,0] - rect[0] # shift contour to roi
        points[:,1] = points[:,1] - rect[1]
        delta = np.mean(np.abs(laplacian * parking_mask[ind]))
        if(delta > 1.8):    # Bỏ qua space trống
            delta_list.append(delta)
        sum_up = sum_up + delta
        
avg = sum_up/len(parking_data)
med = statistics.median(delta_list)
print("mean: ", avg)
print("median: ", med)

<ipython-input-16-66529c9b6fef>:15: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  parking_data = yaml.load(stream)


StatisticsError: no median for empty data

## Kết luận
Hệ thống đề xuất một phương pháp hiệu quả để quản lý hệ thống bãi đậu xe bằng cách sử dụng máy ảnh được đặt ở các cột đèn. Độ chính xác của hệ thống phụ thuộc vào giá trị Threshold, thuật toán theo dõi chuyển động dựa trên background subtraction và quan trọng nhất là sức mạnh của classifier. Thách thức phải đối mặt là để xác định chuyển động trong một ROI nhỏ khi có tồn tại khả năng có nhiễu. Ví dụ, máy ảnh được đặt gần cầu thang và reflection của những người đi bộ xuống cầu thang đã được reflect trong video đầu vào được cung cấp bởi máy ảnh. Nó trở nên khó khăn để giảm tác động của nhiễu và phát hiện được các chuyển động nhỏ. Bên cạnh đó, Haar classifier đã thực hiện tốt mục tiêu đặt ra của đề tài nhưng thực hiện classifier dựa trên Convolution Neural Networks sẽ làm tăng độ chính xác tổng thể của hệ thống một cách đáng kể. Đây là phạm vi công việc trong tương lai của dự án này. Tuy nhiên với hiện tại thì mức độ chính xác của hệ thống là rất tốt với độ chính xác hơn 85% với mô hình bãi đỗ xe vừa-nhỏ và hơn 62.5% với mô hình bãi độ xe lớn.

## Tham khảo

### 1. Video 
-https://www.youtube.com/watch?v=bPeGC8-PQJg

-https://www.youtube.com/watch?v=Dg-4MoABv4I


### 2. Codes
-https://github.com/eladj/detectParking

-http://www.pyimagesearch.com/2015/11/09/pedestrian-detection-opencv/

### 3. Papers
[1] M. I. et al, “Car park system: A review of smart parking system and its technology,” Information Technology Journal, 2009.
In this study, various types of smart parking systems with their pros and cons are presented.

[2] N. True, “Vacant parking space detection in static images,” 2007. http://cseweb.ucsd.edu/classes/wi07/cse190-a/reports/ntrue.pdf